# MATH50003 Numerical Analysis: Problem 3

This problem sheet explores implementation of triangular solves,
supporting a matrix with two super-diagonals, as well as
permutation and Householder reflections that can be applied to a vector in
$O(n)$ complexity.

In [1]:
using LinearAlgebra, Test

# We will override these functions below
import Base: getindex, setindex!, size, *, \

## 1. Dense Matrices

**Problem 1.1** Show that `A*x` is not
implemented as `mul(A, x)` from the lecture notes
by finding a `Float64` example  where the bits do not match.




## 2. Triangular Matrices

**Problem 2.1** Complete the following functions for solving linear systems with
triangular systems by implementing back and forward-substitution:

In [2]:
function ldiv(U::UpperTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using back-substitution

    
    for k = n:-1:1  # start with k=n, then k=n-1, ...
        r = b[k]  # dummy variable
        for j = k+1:n
            r -= U[k,j]*x[j] # equivalent to r = r-U[k,j]*x[j]
        end
        x[k] = r/U[k,k]
    end
    x
end

function ldiv(U::LowerTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using forward-substitution
end

ldiv (generic function with 2 methods)

**Problem 2.2‚ãÜ** Given $ùê± ‚àà ‚Ñù^n$, find a lower triangular matrix of the form
$$
L = I - 2 ùêØ ùêû_1^‚ä§
$$
such that:
$$
L ùê± = x_1 ùêû_1.
$$
What does $Lùê≤$ equal if $ùê≤  ‚àà ‚Ñù^n$ satisfies $y_1 = ùêû_1^‚ä§ ùê≤ = 0$?

## 3. Banded matrices

**Problem 3.1** Complete the implementation of `UpperTridiagonal` which represents a banded matrix with
bandwidths $(l,u) = (0,2)$:

In [3]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    # TODO: return U[k,j]
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j > k+2
        error("Cannot modify off-band")
    end

    # TODO: modify d,du,du2 so that U[k,j] == v

    U # by convention we return the matrix
end

setindex! (generic function with 90 methods)

**Problem 3.2** Complete the following implementations of `*` and `\` for `UpperTridiagonal` so that
they take only $O(n)$ operations.

In [4]:
function *(U::UpperTridiagonal, x::AbstractVector)
    T = promote_type(eltype(U), eltype(x)) # make a type that contains both the element type of U and x
    b = zeros(T, size(U,1)) # returned vector
    # TODO: populate b so that U*x == b (up to rounding)
end

function \(U::UpperTridiagonal, b::AbstractVector)
    T = promote_type(eltype(U), eltype(b)) # make a type that contains both the element type of U and b
    x = zeros(T, size(U,2)) # returned vector
    # TODO: populate x so that U*x == b (up to rounding)
end

\ (generic function with 136 methods)

## 4. Permutations

**Problem 4.1‚ãÜ** What are the permutation matrices corresponding to the following permutations?
$$
\begin{pmatrix}
1 & 2 & 3 \\
3 & 2 & 1
\end{pmatrix}, \begin{pmatrix}
1 & 2 & 3 & 4 & 5 & 6\\
2 & 1 & 4 & 3 & 6 & 5
\end{pmatrix}.
$$


**Problem 4.2** Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` and such that `*` takes $O(n)$ operations.

In [5]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

size(P::PermutationMatrix) = (length(P.p),length(P.p))

function getindex(P::PermutationMatrix, k::Int, j::Int)
    # TODO: Return P[k,j]
    P.p[k] == j ? 1 : 0 
end

function *(P::PermutationMatrix, x::AbstractVector)
    # TODO: permute the entries of x
    x[P.p]
end

# If your code is correct, this "unit test" will succeed
p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]

Test Passed
  Expression: P == (I(5))[p, :]
   Evaluated: [1 0 ‚Ä¶ 0 0; 0 0 ‚Ä¶ 1 0; ‚Ä¶ ; 0 0 ‚Ä¶ 0 1; 0 0 ‚Ä¶ 0 0] == sparse([1, 3, 5, 2, 4], [1, 2, 3, 4, 5], Bool[1, 1, 1, 1, 1], 5, 5)

## 5. Orthogonal matrices

**Problem 5.1‚ãÜ** Show that orthogonal matrices preserve the 2-norm of vectors:
$$
\|Q ùêØ\| = \|ùêØ\|.
$$

**SOLUTION**
$$
\|Q ùêØ\|^2 = (Q ùêØ)^‚ä§ Q ùêØ = ùêØ^‚ä§ Q^‚ä§ Q ùêØ = ùêØ^‚ä§  ùêØ = \|ùêØ\|^2
$$
‚àé

**Problem 5.2‚ãÜ** Show that the eigenvalues $Œª$ of an orthogonal matrix $Q$ are
on the unit circle: $|Œª| = 1$.

**SOLUTION**
Let $ùêØ$ be a unit eigenvector corresponding to $Œª$: $Q ùêØ = Œª ùêØ$ with $\|ùêØ\| = 1$. Then
$$
1 = \| ùêØ \| = \|Q ùêØ \| =  \| Œª ùêØ \| = |Œª|.
$$
‚àé

**Problem 5.3‚ãÜ** Explain why an orthogonal matrix $Q$ must be equal to $I$ if all its eigenvalues are 1.


**Problem 5.4** Complete the implementation of a type representing
reflections that supports `Q[k,j]` and such that `*` takes $O(n)$ operations.

In [6]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

size(Q::Reflection) = (length(Q.v),length(Q.v))

function getindex(Q::Reflection, k::Int, j::Int)
    # TODO: Return Q[k,j]
    (k == j) - 2Q.v[k]*Q.v[j]
end

function *(P::Reflection, x::AbstractVector)
    # TODO: permute the entries of x
    x - 2*Q.v * dot(Q.v,x)
end

# If your code is correct, these "unit tests" will succeed
x = randn(5)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q*v ‚âà -v
@test Q'Q ‚âà I

Test Passed
  Expression: Q' * Q ‚âà I
   Evaluated: [0.9999999999999998 1.0842021724855044e-18 ‚Ä¶ 1.3877787807814457e-16 1.3877787807814457e-16; 1.0842021724855044e-18 1.0 ‚Ä¶ -1.3010426069826053e-18 -1.3010426069826053e-18; ‚Ä¶ ; 1.3877787807814457e-16 -1.3010426069826053e-18 ‚Ä¶ 0.9999999999999998 -1.1102230246251565e-16; 1.3877787807814457e-16 -1.3010426069826053e-18 ‚Ä¶ -1.1102230246251565e-16 0.9999999999999998] ‚âà UniformScaling{Bool}(true)

**Problem 5.5** Complete the following implementation of a Housholder reflection  so that the
unit tests pass.

In [7]:
function householderreflection(s::Bool, x::AbstractVector)
    # TODO: implement Householder reflection, returning a Reflection
end

x = randn(5)
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ‚âà norm(x)*[1; zeros(length(x)-1)]

QÃÉ = householderreflection(false, x)
@test QÃÉ isa Reflection
@test QÃÉ*x ‚âà -norm(x)*[1; zeros(length(x)-1)]

Test Failed at In[7]:7
  Expression: Q isa Reflection
   Evaluated: nothing isa Reflection


LoadError: [91mThere was an error during testing[39m